In [3]:
# Example of what I think Vinny's method is

In [4]:
from queue import Queue
import time
import threading

In [102]:
# Threaded logging object. Messages are added by other things and periodically new messages
# are printed.
class Logger(object):
    def __init__(self):
        self.messages = []
        self.print_start_pos = 0
    def add(self,message):
        self.messages.append(message)
    def reset(self):
        self.messages = []
    def __str__(self):
        return "\n".join(self.messages)
    def run(self):
        while True:
            if stop_threads:
                break
            time.sleep(0.01)
            for m in  self.messages[self.print_start_pos:]:
                print(m)
            self.print_start_pos = len(self.messages)

In [191]:
SMALL_TIME = 1e-10

class Scan(object):
    # Simple object to store a pretend scan
    def __init__(self,scan_id,start_time,end_time,scan_type):
        self.start_time = start_time # time the scan started
        self.end_time = end_time # time it ended
        self.scan_type = scan_type # type: method, ms1, ms2
        self.scan_id = scan_id # id (provided by command)
    def __str__(self):
        return "{} scan ({}) starting and end time: {:.3f} -> {:.3f}".format(self.scan_type,self.scan_id,self.start_time,self.end_time)
    
class Command(object):
    # Command sent from controller to MS
    def __init__(self,scan_type,scan_id,submit_time):
        self.scan_type = scan_type # method, ms1, or ms2
        self.scan_id = scan_id # ID
        self.submit_time = submit_time # time this command was submitted -- need this to stop the MS seeing it before it is really there
    def __str__(self):
        return "Command {} type = {}".format(self.scan_id,self.scan_type)

class MS(object):
    # MS
    def __init__(self,command_queue,scan_queue):
        self.time = 0 # the time w.r.t MS
        self.ms1_scan_duration = 0.1 # duration of an ms1
        self.ms2_scan_duration = 0.04 # duration ms2
        self.method_scan_duration = 0.12 # duration method
        self.command_q = command_queue # queue from which commands will appear
        self.scan_q = scan_queue # queue to put scans in
        logger.add("Initialised MS")
        
    def generate_method_scan(self):
        # method to generate a method scan
        end_time = self.time + self.method_scan_duration
        scan = Scan(0,self.time,end_time,"method") # they all have ID = 0.
        self.time = end_time
        self.scan_q.put(scan)
        logger.add("MS:\tAdded method scan {} (time = {:.3f})".format(str(scan),self.time))
    
    def _get_duration(self,scan_type):
        if scan_type == "method":
            duration = self.method_scan_duration
        elif scan_type == "ms1":
            duration = self.ms1_scan_duration
        elif scan_type == "ms2":
            duration = self.ms2_scan_duration
        return duration

    def click(self):
        if self.command_q.empty():
            # easy case, q is empty, make method scan
            self.generate_method_scan()
        else:
            # otherwise, get a command from the Q
            command = self.command_q.get()
            while self.time < command.submit_time:
                # if the MS time is behind the time this command was submitted,
                # keep generating method scans
                self.generate_method_scan()
            scan_type = command.scan_type
            scan_id = command.scan_id
            # work out the duration depending on the type of scan
            duration = self._get_duration(scan_type)
            # make the scan
            scan = Scan(scan_id,self.time,self.time+duration,scan_type)
            self.scan_q.put(scan)
            self.time += duration
            logger.add("MS:\tAdded custom scan {} (local time = {:.3f})".format(str(scan),self.time))


            
            
class Controller(object):
    # Simple controller object
    def __init__(self,command_q,scan_q,N = 10):
        self.command_q = command_q
        self.scan_q = scan_q
        self.N = N # for top N
        self.next_scan_id = 1 # first scan will have ID 1 (method scans are always ID = 0)
        self.processing_time = 1 # time it takes to process an MS1 and decide which precursors to fragment
        # time it takes to submit a scan.
        # i.e. controller takes processing_time to decide what to fragment
        # and then it takes submit_delay to actually submit each one (avoids them all having the same time)
        self.submit_delay = 0.001 
        self.time = 0
        logger.add("Initialised controller")
        
    def first_scan(self):
        # First scan is MS1. Need this because my controller schedules the next MS1 at the end of the set of MS2
        self.time += self.submit_delay
        command = Command("ms1",self.next_scan_id,self.time)
        self.next_scan_id += 1
        self.command_q.put(command)
        logger.add("CONT:\tAdded {} to command q (time = {:.3f})".format(str(command),self.time))

    
    def click(self):
        if self.scan_q.empty():
            # do nothing
            pass
        else:
            # process the scan that has arrived
            self.process(self.scan_q.get())
    
    def process(self,scan):
        self.time = scan.end_time + SMALL_TIME
        logger.add("CONT:\tReceived scan {} (time = {:.3f})".format(str(scan),self.time))
        if not scan.scan_type == "ms1":
            # if it isn't an MS1, ignore
            # but forward wind time
            pass
        else:
            # generate N 
            self.time = scan.end_time + self.processing_time # simulating working out whiich N
            for n in range(self.N):
                self.time += self.submit_delay
                command = Command("ms2",self.next_scan_id,self.time)
                self.next_scan_id += 1
                self.command_q.put(command)
                logger.add("CONT:\tAdded {} to command q (time = {:.3f})".format(str(command),self.time))
            # Schedule the MS1 at the end
            self.time += self.submit_delay
            command = Command("ms1",self.next_scan_id,self.time)
            self.next_scan_id += 1
            self.command_q.put(command)
            logger.add("CONT:\tAdded {} to command q (time = {:.3f})".format(str(command),self.time))
    

In [133]:
# make the queues
scan_queue = Queue()
command_queue = Queue()

In [134]:
# logging thread will stop when stop_threads = True
stop_threads = False

# make a logger, put it on a thread, and start it
logger = Logger()
log_thread = threading.Thread(target=logger.run)
log_thread.start()

# make MS and controller
ms = MS(command_queue,scan_queue)
controller = Controller(command_queue,scan_queue)
controller.first_scan()

# so we don't go on forever
max_steps = 1000
step = 0
while step < max_steps:
    step += 1
    ms_time = ms.time
    c_time = controller.time
    if ms_time < c_time:
        # ms is behind, it should go first
        ms.click()
    else:
        # funky logic
        # if there are scans in the queue, click the controller
        # it *must* be behind in time
        if not scan_queue.empty():
            controller.click()
        else:
            # nothing can happen until the MS has done something
            # so we can increase controller time to just after ms time
            controller.time = ms.time + SMALL_TIME
            
stop_threads = True

Initialised MS
Initialised controller
CONT:	Added Command 1 type = ms1 to command q (time = 0.001)
MS:	Added method scan method scan (0) starting at time: 0 (time = 0.12)
MS:	Added custom scan ms1 scan (1) starting at time: 0.12 (local time = 0.22)
CONT:	Received scan method scan (0) starting at time: 0 (time = 0.12000000009999999)
CONT:	Received scan ms1 scan (1) starting at time: 0.12 (time = 0.2200000001)
CONT:	Added Command 2 type = ms2 to command q (time = 1.2209999999999999)
CONT:	Added Command 3 type = ms2 to command q (time = 1.2219999999999998)
CONT:	Added Command 4 type = ms2 to command q (time = 1.2229999999999996)
CONT:	Added Command 5 type = ms2 to command q (time = 1.2239999999999995)
CONT:	Added Command 6 type = ms2 to command q (time = 1.2249999999999994)
CONT:	Added Command 7 type = ms2 to command q (time = 1.2259999999999993)
CONT:	Added Command 8 type = ms2 to command q (time = 1.2269999999999992)
CONT:	Added Command 9 type = ms2 to command q (time = 1.22799999999999

## Notes

- Output is confusing because it is in the order that the events happen, which isn't necessarily time order...
- Seems to work though, as a bit of a stop-gap

In [209]:
class ThreadMS(MS):
    # extend to replace the click method with a run method
    def run(self):
        while True:
            if stop_threads:
                break
            # check the queue
            if self.command_q.empty():
                self.generate_method_scan()
            else:
                command = self.command_q.get()
                start_time = time.time() - INIT_TIME
                duration = self._get_duration(command.scan_type)
                logger.add("MS:\tStarting custom scan {} (current time = {:.3f})".format(command.scan_type,time.time() - INIT_TIME))
                time.sleep(duration) # making the scan
                scan = Scan(command.scan_id,start_time,time.time() - INIT_TIME,command.scan_type)
                self.scan_q.put(scan)
                logger.add("MS:\tSending {} (current time = {:.3f})".format(str(scan),time.time() - INIT_TIME))
    def generate_method_scan(self):
        # override
        start_time = time.time() - INIT_TIME
        duration = self.method_scan_duration
        logger.add("MS:\tStarting method scan (current time = {:.3f})".format(time.time() - INIT_TIME))
        time.sleep(duration)
        scan = Scan(0,start_time,time.time() - INIT_TIME,"method")
        self.scan_q.put(scan)
        logger.add("MS:\tSending {} (current time = {:.3f})".format(str(scan),time.time() - INIT_TIME))

        
        
                
                
class ThreadCont(Controller):
    def run(self):
        while True:
            if stop_threads:
                break
            if self.scan_q.empty():
                pass
            else:
                scan = self.scan_q.get()
                logger.add("CONT:\tReceived scan {} (current time = {:.3f})".format(str(scan),time.time() - INIT_TIME))
                if not scan.scan_type == 'ms1':
                    pass
                else:
                    time.sleep(self.processing_time)
                    for n in range(self.N):
                        time.sleep(self.submit_delay)
                        command = Command("ms2",self.next_scan_id,time.time() - INIT_TIME)
                        self.next_scan_id += 1
                        self.command_q.put(command)
                        logger.add("CONT:\tAdded {} to command q (current time = {:.3f})".format(str(command),time.time() - INIT_TIME))
                    # Schedule the MS1 at the end
                    time.sleep(self.submit_delay)
                    command = Command("ms1",self.next_scan_id,time.time() - INIT_TIME)
                    self.next_scan_id += 1
                    self.command_q.put(command)
                    logger.add("CONT:\tAdded {} to command q (current time = {:.3f})".format(str(command),time.time() - INIT_TIME))
    
                    
                    
        

In [210]:
# make the queues
scan_queue = Queue()
command_queue = Queue()
stop_threads = False
INIT_TIME = time.time()
logger = Logger()
log_thread = threading.Thread(target = logger.run)
log_thread.start()
ms = ThreadMS(command_queue,scan_queue)
c = ThreadCont(command_queue,scan_queue,N = 3)
c.processing_time = 0.2 # make it faster
c.first_scan()
ms_thread = threading.Thread(target = ms.run)
ms_thread.start()
c_thread = threading.Thread(target = c.run)
c_thread.start()



time.sleep(5) # run for five seconds
stop_threads = True


Initialised MS
Initialised controller
CONT:	Added Command 1 type = ms1 to command q (time = 0.001)
MS:	Starting custom scan ms1 (current time = 0.001)
MS:	Sending ms1 scan (1) starting and end time: 0.001 -> 0.111 (current time = 0.111)
MS:	Starting method scan (current time = 0.111)
CONT:	Received scan ms1 scan (1) starting and end time: 0.001 -> 0.111 (current time = 0.111)
MS:	Sending method scan (0) starting and end time: 0.111 -> 0.236 (current time = 0.236)
MS:	Starting method scan (current time = 0.236)
CONT:	Added Command 2 type = ms2 to command q (current time = 0.313)
CONT:	Added Command 3 type = ms2 to command q (current time = 0.315)
CONT:	Added Command 4 type = ms2 to command q (current time = 0.316)
CONT:	Added Command 5 type = ms1 to command q (current time = 0.317)
CONT:	Received scan method scan (0) starting and end time: 0.111 -> 0.236 (current time = 0.317)
MS:	Sending method scan (0) starting and end time: 0.236 -> 0.363 (current time = 0.364)
MS:	Starting custom sc

MS:	Sending method scan (0) starting and end time: 2.629 -> 2.750 (current time = 2.750)
MS:	Starting method scan (current time = 2.750)
CONT:	Added Command 22 type = ms2 to command q (current time = 2.831)
CONT:	Added Command 23 type = ms2 to command q (current time = 2.833)
CONT:	Added Command 24 type = ms2 to command q (current time = 2.834)
CONT:	Added Command 25 type = ms1 to command q (current time = 2.836)
CONT:	Received scan method scan (0) starting and end time: 2.629 -> 2.750 (current time = 2.836)
MS:	Sending method scan (0) starting and end time: 2.750 -> 2.878 (current time = 2.878)
MS:	Starting custom scan ms2 (current time = 2.878)
CONT:	Received scan method scan (0) starting and end time: 2.750 -> 2.878 (current time = 2.878)
MS:	Sending ms2 scan (22) starting and end time: 2.878 -> 2.927 (current time = 2.927)
MS:	Starting custom scan ms2 (current time = 2.927)
CONT:	Received scan ms2 scan (22) starting and end time: 2.878 -> 2.927 (current time = 2.927)
MS:	Sending ms

In [180]:
stop_threads = True